	# Author: Alexander Staub
	## Last changed: 2025.06.23
	## Purpose: Using the chartmetric IDs to get song level metadata post spotify data collection


In [1]:
#installing packages
import time
import requests
import logging
import pandas as pd
from pprint import pprint
from datetime import datetime
import json

In [ ]:
#--- Cell 1: Configuration ---

NUM_WORKERS = 3  # Must match the number of workers you ran
# --- This should point to the directory containing your "part_1", "part_2", etc. folders ---
METADATA_OUTPUT_DIR = "//bigdata.wu.ac.at/delpero/Data_alexander/data/raw_data/chartmetric/chartmetric_chars/"
# --- This is the path for your final, combined output file. ---
FINAL_OUTPUT_FILE = "//bigdata.wu.ac.at/delpero/Data_alexander/data/raw_data/chartmetric/chartmetric_chars/complete_charmetric_chars.json"


In [ ]:
# --- Cell 2: Combine Worker Results ---
all_responses = []
print("Starting to merge results from all workers...")

for i in range(NUM_WORKERS):
    part_number = i + 1
    # --- CHANGE: The path is dynamically constructed to find each worker's checkpoint file. ---
    file_path = os.path.join(METADATA_OUTPUT_DIR, f"part_{part_number}", "song_chars_checkpoint.json")
    
    if os.path.exists(file_path):
        print(f"Loading {file_path}...")
        with open(file_path, "r") as f:
            data = json.load(f)
            # Use extend to add all items from the loaded list to the master list
            all_responses.extend(data)
    else:
        print(f"WARNING: Could not find result file for worker {part_number} at {file_path}")

print(f"\nSuccessfully merged a total of {len(all_responses)} responses.")


# Code to retreive the song metadata from chartmetrics

- create the get request
- run the loop over each chartmetric id
- save the response for later parsing

In [8]:
# --- Function to Retrieve song characteristics from Chartmetric ID ---
def get_songchars_ids(chartmetric_id):
    endpoint = f"/api/track/{chartmetric_id}"
    try:
        response = get_request(endpoint)
        logging.info(f"Successfully retrieved song chars for Chartmetric ID {chartmetric_id}")
    except Exception as e:
        logging.error(f"Failed to get song chars for Chartmetric ID {chartmetric_id}: {e}")
        return None

    # The API response (a dictionary) is returned as is
    song_chars = response
    return song_chars

In [11]:
# --- Step 1: Loop over the DataFrame to retrieve API responses ---

# --- Main Processing with Checkpointing & DYNAMIC Throttling ---
TIME_PER_REQUEST = 0.28

song_chars_responses = []  # List to store API responses for each Chartmetric ID.
checkpoint_interval = 100  # Save a checkpoint every 100 processed rows.
checkpoint_file = "//bigdata.wu.ac.at/delpero/Data_alexander/data/incidental/chartmetric/chartmetric_track_id_checkpoints/chartmetric_metadata/song_chars_checkpoint.json"  # Checkpoint file for responses.

for idx, row in chartmetric_ids_spotify.iterrows():
    # Record the start time of the loop
    loop_start_time = time.time()

    chartmetric_id = row.get("chartmetric_ids")
    print(f"Processing row {idx}: Chartmetric ID = {chartmetric_id}")
    logging.info(f"Processing row {idx}: Chartmetric ID = {chartmetric_id}")
    
    if pd.isnull(chartmetric_id):
        print(f"Row {idx} has no Chartmetric ID. Skipping.")
        logging.info(f"Row {idx} has no Chartmetric ID. Skipping.")
        continue
    
    try:
        song_chars = get_songchars_ids(chartmetric_id)
    except Exception as e:
        print(f"Error processing Chartmetric ID {chartmetric_id} at row {idx}: {e}")
        logging.error(f"Error processing Chartmetric ID {chartmetric_id} at row {idx}: {e}")
        song_chars = None
    
    # Append the response (or None) to our list.
    song_chars_responses.append(song_chars)
    
    print(f"Row {idx} processed: Chartmetric ID = {chartmetric_id}")
    logging.info(f"Processed row {idx}: Chartmetric ID = {chartmetric_id}")
    
    # --- DYNAMIC SLEEP LOGIC ---
    # REMOVED: time.sleep(0.3)

    # Calculate how long the API call and processing took
    loop_end_time = time.time()
    elapsed_time = loop_end_time - loop_start_time

    # Calculate the remaining time to sleep to hit the target rate
    sleep_duration = TIME_PER_REQUEST - elapsed_time

    # If the request was fast, sleep for the remaining time.
    # If the request was slow (elapsed_time > TIME_PER_REQUEST), don't sleep at all.
    if sleep_duration > 0:
        time.sleep(sleep_duration)
    
    # Save a checkpoint periodically.
    if idx % checkpoint_interval == 0 and idx > 0:
        with open(checkpoint_file, "w") as f:
            json.dump(song_chars_responses, f, indent=2)
        print(f"Checkpoint saved at row {idx}")
        logging.info(f"Checkpoint saved at row {idx}")

Processing row 0: Chartmetric ID = 12486264.0
Row 0 processed: Chartmetric ID = 12486264.0
Processing row 1: Chartmetric ID = 12486289.0
Row 1 processed: Chartmetric ID = 12486289.0
Processing row 2: Chartmetric ID = 16790822.0
Row 2 processed: Chartmetric ID = 16790822.0
Processing row 3: Chartmetric ID = 10926819.0
Row 3 processed: Chartmetric ID = 10926819.0
Processing row 4: Chartmetric ID = 11060010.0
Row 4 processed: Chartmetric ID = 11060010.0
Processing row 5: Chartmetric ID = 11053742.0
Row 5 processed: Chartmetric ID = 11053742.0
Processing row 6: Chartmetric ID = 11062252.0
Row 6 processed: Chartmetric ID = 11062252.0
Processing row 7: Chartmetric ID = 93916438.0
Row 7 processed: Chartmetric ID = 93916438.0
Processing row 8: Chartmetric ID = 34844230.0
Row 8 processed: Chartmetric ID = 34844230.0
Processing row 9: Chartmetric ID = 100099608.0
Row 9 processed: Chartmetric ID = 100099608.0
Processing row 10: Chartmetric ID = 11037935.0
Row 10 processed: Chartmetric ID = 110379

In [69]:
# define a function to extract necessary information from the search output
def extract_song_info(search_output):
    # Extract the main object
    obj = search_output.get('obj', {})
    
    # Artist: take first artist if available
    if obj.get('artists') and len(obj['artists']) > 0:
        artist = obj['artists'][0]
        artist_id = artist.get('id', None)
        artist_name = artist.get('name', None)
        artist_label = artist.get('label', None)
        artist_booking_agent = artist.get('booking_agent', None)
        artist_general_manager = artist.get('general_manager', None)
    else:
        artist_id = artist_name = artist_label = artist_booking_agent = artist_general_manager = None

    # Albums: select the album with the earliest release date
    if obj.get('albums') and len(obj['albums']) > 0:
        def parse_date(album):
            try:
                return datetime.strptime(album.get('release_date', ''), '%Y-%m-%d')
            except Exception:
                return datetime.max
        sorted_albums = sorted(obj['albums'], key=parse_date)
        earliest_album = sorted_albums[0]
        album_id = earliest_album.get('id', None)
        album_name = earliest_album.get('name', None)
        album_release_date = earliest_album.get('release_date', None)
        album_label = earliest_album.get('label', None)
    else:
        album_id = album_name = album_release_date = album_label = None

    # Use a pipe '|' as delimiter for multiple values
    delimiter = '|'
    
    # Moods: concatenate mood names
    if obj.get('moods') and len(obj['moods']) > 0:
        moods = delimiter.join([m.get('name', '') for m in obj['moods']])
    else:
        moods = None
    
    # Activities: concatenate activity names
    if obj.get('activities') and len(obj['activities']) > 0:
        activities = delimiter.join([a.get('name', '') for a in obj['activities']])
    else:
        activities = None
    
    # Songwriters: concatenate songwriter names
    if obj.get('songwriters') and len(obj['songwriters']) > 0:
        songwriters = delimiter.join(obj['songwriters'])
    else:
        songwriters = None
    
    # songwriterIds is not present in the example so we assign None
    songwriterIds = None

    # Create a one-row DataFrame with the desired columns
    data = {
        'chartmetric_ids': obj.get('id', None),
        'Name': obj.get('name', None),
        'Composer_name': obj.get('composer_name', None),
        'Artist_id': artist_id,
        'Artist_name': artist_name,
        'Artist_label': artist_label,
        'Artist_booking_agent': artist_booking_agent,
        'Artist_general_manager': artist_general_manager,
        'Albums_id': album_id,
        'Albums_name': album_name,
        'Albums_release_date': album_release_date,
        'Albums_label': album_label,
        'Tags': obj.get('tags', None),
        'Moods': moods,
        'Activities': activities,
        'Songwriters': songwriters,
        'songwriterIds': songwriterIds,
        'Tempo': obj.get('tempo', None),
        'Duration_ms': obj.get('duration_ms', None)
    }
    
    return pd.DataFrame([data])

In [56]:
# --- Step 2: Parse the Collected API Responses ---
# Assume the extraction function 'extract_song_info' is defined (as provided previously).
extracted_rows = []
for resp in song_chars_responses:
    if resp is not None:
        # Extract song information from the response.
        extracted_df = extract_song_info(resp)
        extracted_rows.append(extracted_df)

if extracted_rows:
    song_chars_extracted = pd.concat(extracted_rows, ignore_index=True)
else:
    song_chars_extracted = pd.DataFrame()

C:\Users\User\AppData\Local\Temp\ipykernel_7044\3021864179.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  song_chars_extracted = pd.concat(extracted_rows, ignore_index=True)


In [ ]:
# --- Step 3: Join the Extracted Data to the Original DataFrame ---
# It is assumed that the 'id' column in song_chars_extracted matches the 'id' column in spotify_sample.
merged_song_chars = chartmetric_ids_spotify.merge(song_chars_extracted, on="chartmetric_ids", how="left")

In [ ]:
#safe the final dataframe
# Save as JSON (records-oriented with one JSON object per line)

#sample dataset
merged_song_chars.to_json("Z:/Data_alexander/data/raw_data/chartmetric/chartmetric_chars/sample_charmetric_chars.json", orient="records", lines=True)

#the songs + artist 1980-2000 dataset
# merged_song_chars.to_json("//bigdata.wu.ac.at/delpero/Data_alexander/data/incidental/chartmetric/chartmetric_metadata/all_1980_2000_charmetric_chars.json", orient="records", lines=True)


# The code used to derive an example and extract information

In [26]:
# --- Function to Retrieve song characteristics from Chartmetric ID ---
def get_songchars_ids(chartmetric_id):
    endpoint = f"/api/track/{chartmetric_id}"
    try:
        response = get_request(endpoint)
        # Log the response status code and rate limit headers
        logging.info(f"Successfully retrieved song chars for chartmetric id {chartmetric_id}")
    except Exception as e:
        logging.error(f"Failed to get song chars for chartmetric id {chartmetric_id}: {e}")
        return None

    # Extract the song characteristics from the response
    song_chars = response

    return song_chars

In [34]:
pprint(chartmetric_ids_spotify["chartmetric_ids"][4])

15447513.0


In [ ]:
# trial run with a single ID 

search_output = get_songchars_ids(chartmetric_ids_spotify["chartmetric_ids"][0])


pprint(search_output)

In [40]:
#trial the function with the search output

test_df = extract_song_info(search_output)